#Necessary Imports

In [1]:
from ast import Pass
import math
from itertools import combinations
from math import isclose
import pandas as pd

In [2]:
pip install nltk

In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import wordnet
from nltk.wsd import lesk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import wordnet, pos_tag
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from ir_explain.pairwise.axioms import pairwise

In [ ]:
from ir_explain.pairwise.explain_more import explain_more

#Showcase some features of the ir_explain.pairwise framework example data:

*   TREC-DL-2019







In [8]:
!pip install ir_datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18919 sha256=21a60b6d0b97ddabb9ccf447c1528ebb870059327ba99f1004b25733aa86025f
  Stored in directory: /root/.cache/pip/wheels/1a/d7/91/7ffb991df87e62355d945745035470ba2616aa3d83a250b5f9
  Created wheel for zlib-state: filename=zlib_state-0.1.6-cp310-cp310-linux_x86_64.whl size=21160 sha256=5ccc7c79395437d54674ae24056b77

#Get some query from the list to test and display the text of the query

In [9]:
import ir_datasets
dataset = ir_datasets.load("msmarco-passage/trec-dl-2019")
query_id = "183378"

query_text = None
for query in dataset.queries_iter():
    if query.query_id == query_id:
        query_text = query.text
        break
if query_text is not None:
    print(f"Query ID: {query_id}\nQuery Text: {query_text}")
else:
    print(f"No query found with ID: {query_id}")


[INFO] Please confirm you agree to the MSMARCO data usage agreement found at <http://www.msmarco.org/dataset.aspx>
[INFO] [starting] https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz
[INFO] [finished] https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz: [00:00] [4.28kB] [14.8MB/s]
                                                                                                                   

Query ID: 183378
Query Text: exons definition biology


#Get the top 20 ranked documents for the given query

In [18]:
target_query_id = "183378"

target_query_docs = []

for scoreddoc in dataset.scoreddocs_iter():
    if scoreddoc.query_id == target_query_id:
        target_query_docs.append((scoreddoc.doc_id, scoreddoc.score))
i=1
for doc_id, score in target_query_docs:
    print(f"Rank: {i}, Doc ID: {doc_id}")
    i+=1
    if i==21:
      break


Rank: 1, Doc ID: 7135097
Rank: 2, Doc ID: 8022481
Rank: 3, Doc ID: 7173250
Rank: 4, Doc ID: 8440424
Rank: 5, Doc ID: 719550
Rank: 6, Doc ID: 8458194
Rank: 7, Doc ID: 5210173
Rank: 8, Doc ID: 7221921
Rank: 9, Doc ID: 8073356
Rank: 10, Doc ID: 8079968
Rank: 11, Doc ID: 7248228
Rank: 12, Doc ID: 8507869
Rank: 13, Doc ID: 5259281
Rank: 14, Doc ID: 5260519
Rank: 15, Doc ID: 7275691
Rank: 16, Doc ID: 7278405
Rank: 17, Doc ID: 8529539
Rank: 18, Doc ID: 5286892
Rank: 19, Doc ID: 8139400
Rank: 20, Doc ID: 7315393


#Lets take the top ranked document and test it against 10th ranked document

In [14]:
target_doc_id = "7135097"

doc1 = ""

for doc in dataset.docs_iter():
    if doc.doc_id == target_doc_id:
        doc1 = doc.text
        break

if doc1 is not None:
    print(f"Body of Document ID {target_doc_id}:\n{doc1}")
else:
    print(f"No document found with ID {target_doc_id}")


Body of Document ID 7135097:
Herbivory: Definition & Examples 4:44 Next Lesson. Allometric Growth: Definition & Example; Isometric Growth: Definition & Example; Go to AP Biology - Animal Behavior: Help and Review


In [13]:
target_doc_id = "8079968"

doc2 = ""

for doc in dataset.docs_iter():
    if doc.doc_id == target_doc_id:
        doc2= doc.text
        break

if doc2 is not None:
    print(f"Body of Document ID {target_doc_id}:\n{doc1}")
else:
    print(f"No document found with ID {target_doc_id}")


Body of Document ID 8079968:
Herbivory: Definition & Examples 4:44 Next Lesson. Allometric Growth: Definition & Example; Isometric Growth: Definition & Example; Go to AP Biology - Animal Behavior: Help and Review


# Applying Preprocessing on the docs

In [16]:
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess_string(input_string):

    words = word_tokenize(input_string)

    processed_words = [ps.stem(word) for word in words if word.lower() not in stop_words]

    return ' '.join(processed_words)


# Preprocess the strings
doc1 = preprocess_string(doc1)
doc2 = preprocess_string(doc2)
query_text = preprocess_string(query_text)



#Applying Selected Axioms to see which doc is preffered


*   TFC1: assigns higher scores
to documents that contain a query term more often normalised by document length

*   LB1:If there is a query term t with tf(t, d1) = 0
and tf(t, d2) > 0, then d1 < d2.

*   LNC1:  two documents are compared that have the same term frequency
for all query terms (10%-softening). Then the shorter document is preferred.

*   AND: from a document pair d1 and d2 where only d1 contains all
query terms, d1 is preferred.

*   PROX1-3: The closer the query terms appear to each other in a document, the more relevant the document is.








In [35]:
index_path = ""

# Create an instance of the pairwise class
pairwise_instance = pairwise(query_text, doc1, doc2, index_path)

axiom_classes = ["LNC1", "LB1", "AND", "DIV", "PROX1"]

pairwise_instance.explain(query_text, doc1, doc2, axiom_classes)

,Query,Document 1,Document 2,LNC1,LB1,AND,DIV,PROX1
0,exon definit biolog,herbivori : definit & exa,2 mean nad acronym nad ab,1,0,0,1,0


In [55]:
pairwise_instance = pairwise(query_text, doc1, doc2, index_path)

axiom_classes = ["4*LNC1 + 3*TFC1", "- DIV", "PROX1 + PROX2 + PROX3"]

pairwise_instance.explain(query_text, doc1, doc2, axiom_classes)

,Query,Document 1,Document 2,4*LNC1 + 3*TFC1,- DIV,PROX1 + PROX2 + PROX3
0,exon definit biolog,herbivori : definit & exa,2 mean nad acronym nad ab,1,-1,0


More details on the following axiom -


*   LNC1
*   AND



In [32]:
axiom_name = "LNC1"
explanation = pairwise_instance.explain_details(query_text, doc1, doc2, axiom_name)
print(explanation)

Number of query terms document 1: 2
Number of query terms document 2: 2

Length of document1: 163
Length of document2: 164
None


In [53]:
axiom_name = "AND"
explanation = pairwise_instance.explain_details(query_text, doc1, doc2, axiom_name)
print(explanation)

Query terms not present in document 1: {'exon'}
Query terms not present in document 2: {'exon'}
None
